In [2]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, assemble, QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.compiler import transpile
from qiskit.providers.ibmq.job import job_monitor


In [ ]:
def multiply(num1, num2):
    """
    Description for multiplier can be seen on README file
    """
    ### Prepare QuantumRegisters based on num1, num2 ### 
    num1_qubits = np.floor(np.log2(num1)+1)
    num2_qubits = np.floor(np.log2(num2)+1)
    l1 = int(num1_qubits)
    l2 = int(num2_qubits)
    multiplicand_in = np.binary_repr(num1)
    multiplier_in = np.binary_repr(num2)
    
    if l2 > l1:
        multiplier_in, multiplicand_in = multiplicand_in, multiplier_in
        l2, l1 = l1, l2
    multiplicand = QuantumRegister(l1)
    multiplier = QuantumRegister(l2)
    accumulator = QuantumRegister(l1 + l2)
    cl = ClassicalRegister(l1 + l2)
    d = QuantumRegister(1)
    ### Circuit for our case ### 
    qc = QuantumCircuit(accumulator, multiplier, multiplicand,
    d, cl, name="qc")

    ### Prepare Registers based on num1, num2### 
    qc.x(d)
    for i in range(l1):
        if multiplicand_in[i] == '1':
            qc.x(multiplicand[l1 - i - 1])

        for i in range(l2):
            if multiplier_in[i] == '1':
                qc.x(multiplier[l1 - i - 1])
    
    multiplier_str = '1'

    while(int(multiplier_str) != 0):
        n = len(accumulator) - 1

        ### QFT on the accumulator registers to create the superposition state ###
        for i in range(0, n + 1):
            qc.h(accumulator[n - i])
            for k in range(0, n - i):
                qc.cu1(np.pi / float(2**(k + 1)), accumulator[n - i - (k + 1)], accumulator[n - i])
        ### Rotation ###
        for i in range(0, n + 1):
            l = len(multiplicand)
            for k in range(0, n - i + 1):
                if (n - i - k) > l - 1:
                    pass
                else:
                    qc.cu1(1*np.pi / float(2**(k)), multiplicand[n - i - k], accumulator[n - i])
        ### Inverse QFT ### 
        for i in range(0, n + 1):
            for k in range(0, i):
                qc.cu1(-1 * np.pi / float(2**(i - k)), accumulator[k], accumulator[i])
            qc.h(accumulator[i])

        n = len(multiplier) - 1

        ### QFT on multiplier register ### 
        for i in range(0, n + 1):
            qc.h(multiplier[n - i])
            for k in range(0, n - i):
                qc.cu1(np.pi / float(2**(k + 1)), multiplier[n - i - (k + 1)], multiplier[n - i])
        ### Rotation gates to decrease the number of multiplier by 1 ### 
        for i in range(0, n + 1):
            l = len(d)
            for k in range(0, n - i + 1):
                if (n - i - k) > l - 1:
                    pass
                else:
                    qc.cu1(-1*np.pi / float(2**(k)), d[n - i - k], multiplier[n - i])
        ### Inverse QFT ###
        for i in range(0, n + 1):
            for k in range(0, i):
                qc.cu1(-1 * np.pi / float(2**(i - k)), multiplier[k], multiplier[i])
            qc.h(multiplier[i])
        
        ### measure to see whether the multiplier is 0 or not ###
        for i in range(len(multiplier)):
            qc.measure(multiplier[i], cl[i])
        result = execute(qc, backend=Aer.get_backend('qasm_simulator'),
                        shots=2).result().get_counts(qc.name)
        multiplier_str = list(result.keys())[0]

    qc.measure(accumulator, cl)
    result = execute(qc, backend=Aer.get_backend('qasm_simulator'),
                shots=2).result().get_counts(qc.name)

    print(result)

In [ ]:
def multiplier(num1: int, num2: int):
    """
    Multiplier function I made by myself without copying the results from the past paper
    """
    ### Encoding ###
    qubits1 = int(np.floor(np.log2(num1-1))+1)
    qubits2 = int(np.floor(np.log2(num2))+1)
    num1_bi = np.binary_repr(num1-1)
    num2_bi = np.binary_repr(num2)
    
    ### Define the Quantum Circuit ###
    qc = QuantumCircuit(qubits1 + 2*qubits2,3)
    
    for i in range(qubits1):
        if num1_bi[i] == "1":
            qc.x(i)
    for i in range(qubits2):
        if num2_bi[i] == "1":
            qc.x(qubits1 + i)
            qc.x(qubits1 + qubits2 + i)
    
    ### Apply QFT to the second 5 register ###
    qft = QuantumCircuit(qubits2)
    for qubit in range(qubits2-1):
        qft.h(qubits2-qubit-1)
        for control in range(qubits2-qubit-1):
            qft.cp(np.pi/2**(control + 1), qubits2-qubit-control-2, qubits2-qubit-1)
    qft.h(0)
    for qubit in range(qubits2//2):
        qft.swap(qubit, qubits2-qubit-1)

    qc.append(qft, [qubit+qubits1+qubits2 for qubit in range(qubits2)])

    ### Create the controlled U gate ### 
    qcu = QuantumCircuit(2*qubits2)
    for qubit in range(qubits2):
        for qubit2 in range(qubits2-qubit):
            qcu.cp(np.pi/2**(qubit2), qubit+qubit2, qubits2 + qubit)
    cu_gate = qcu.to_gate().control(1)
    
    ### Apply cu_gate to qc ###
    for qubit in range(qubits1):
        for p in range(qubits1-qubit):
            qc.append(cu_gate, [qubits1-1-qubit]+[qubits1 + i for i in range(2*qubits2)])
    
    ### Apply inverse QFT ###
    iqft = qft.inverse()
    qc.append(iqft, [qubit+qubits1+qubits2 for qubit in range(qubits2)])

    ### Measure the phase ###
    for n in range(qubits2):
        qc.measure(n+qubits1+qubits2, n)
    print(qc)

    ### Run the code on the simulator ###
    backend = Aer.get_backend('aer_simulator')
    shots = 2048
    transpiled_qc = transpile(qc, backend, optimization_level=3)
    job = backend.run(transpiled_qc, shots=shots)
    job_monitor(job)
    counts = job.result().get_counts()
    return counts

In [ ]:
def multiplier2(num1: int, num2: int) -> int:
    """
    """
    ### Encoding and setting the quantum circuit###
    num1_qubits = np.floor(np.log2(num1)+1)
    num2_qubits = np.floor(np.log2(num2)+1)
    qubits1 = int(num1_qubits)
    qubits2 = int(num2_qubits)
    num1_bi = np.binary_repr(num1)
    num2_bi = np.binary_repr(num2)
    qc = QuantumCircuit(2*(qubits1 + qubits2)+1, qubits1 + qubits2+1)
    for i in range(qubits1):
        if num1_bi[i] == "1":
            qc.x(i)
    for i in range(qubits2):
        if num2_bi[i] == "1":
            qc.x(qubits1 + i)
    ### Applying QFT to the input of zeros ###
    qft = QuantumCircuit(qubits1 + qubits2+1, name="QFT")
    for qubit in range(qubits1 + qubits2):
        qft.h(qubits1 + qubits2-qubit)
        for control in range(qubits1 + qubits2-qubit-1):
            qft.cp(np.pi/2**(control + 1), qubits1 + qubits2-qubit-control-1, qubits1 + qubits2-qubit)
        qft.barrier()
    qft.h(0)
    for qubit in range((qubits1 + qubits2+1)//2):
        qft.swap(qubit, qubits1 + qubits2-qubit)
    print(qft)
    qc.append(qft, [qubit+qubits1+qubits2 for qubit in range(qubits1 + qubits2+1)])
    
    ### Create the rotation gate ###
    for i in range(qubits1):
        
        rot_gate = QuantumCircuit(qubits1 + 2*qubits2+1, name="controlled rotation")
        for k in range(qubits2):
            rot_gate.crz(np.pi/2**(k+1), k+i, qubits2+i)
            
        for qubit in range(qubits1):
            for control in range(qubits1-qubit):
                rot_gate.crz(np.pi/2**(control), qubit+control, i + qubits1+ qubit+1)
            
        
        crot_gate = rot_gate.to_gate().control(1)
        qc.append(crot_gate, [qubits1 - i -1] + [qubits1 +qubit for qubit in range(qubits1 + 2*qubits2+1)])
        print(rot_gate)
        
    ### Apply inverse QFT ###
    iqft = qft.inverse()
    qc.append(iqft, [qubit+qubits1+qubits2 for qubit in range(qubits1 + qubits2+1)])
    
    
    ### Measure the phase ###
    for n in range(qubits1 + qubits2+1):
        qc.measure(n+qubits1+qubits2, n)
    
    print(qc)
    ### Run the circuit on simulator ### 
    backend = Aer.get_backend('aer_simulator')
    shots = 2048
    transpiled_qc = transpile(qc, backend, optimization_level=3)
    job = backend.run(transpiled_qc, shots=shots)
    job_monitor(job)
    counts = job.result().get_counts()
    return counts